# Pipeline End-to-End untuk Klasifikasi Ikan menggunakan CNN
## 1. Pengumpulan & Pembersihan Data dengan Pandas
Pertama, kita perlu menyiapkan dan mengeksplorasi dataset yang tersedia.

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

# Tentukan path dataset
base_dir = "FishimgDataset"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")
test_dir = os.path.join(base_dir, "test")

# Dapatkan nama kelas (jenis ikan)
classes = sorted(os.listdir(train_dir))
num_classes = len(classes)
print(f"Jumlah kelas ikan: {num_classes}")
print(f"Kelas ikan: {classes}")

# Hitung jumlah gambar per kelas
for class_name in classes:
    train_samples = len(os.listdir(os.path.join(train_dir, class_name)))
    val_samples = len(os.listdir(os.path.join(val_dir, class_name)))
    test_samples = len(os.listdir(os.path.join(test_dir, class_name)))
    print(f"{class_name}: {train_samples} training, {val_samples} validation, {test_samples} test")

ModuleNotFoundError: No module named 'tensorflow'

In [15]:
pip install tensorflow


  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/376.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/376.0 MB 5.6 MB/s eta 0:01:08
   ---------------------------------------- 1.8/376.0 MB 4.0 MB/s eta 0:01:33
   ---------------------------------------- 2.6/376.0 MB 4.2 MB/s eta 0:01:29
   ---------------------------------------- 3.4/376.0 MB 3.9 MB/s eta 0:01:35
   ---------------------------------------- 4.2/376.0 MB 3.9 MB/s eta 0:01:36
    ----------------------------

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Definisi lokasi dataset
base_dir = "FishimgDataset"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")
test_dir = os.path.join(base_dir, "test")

# Mendapatkan informasi kelas
classes = sorted(os.listdir(train_dir))
num_classes = len(classes)
print(f"Jumlah kelas ikan: {num_classes}")
print(f"Kelas ikan: {classes}")

# Menyimpan statistik data dalam dataframe
stats = []
for class_name in classes:
    train_samples = len(os.listdir(os.path.join(train_dir, class_name)))
    val_samples = len(os.listdir(os.path.join(val_dir, class_name)))
    test_samples = len(os.listdir(os.path.join(test_dir, class_name)))
    stats.append({
        'Spesies': class_name,
        'Train': train_samples,
        'Validation': val_samples,
        'Test': test_samples,
        'Total': train_samples + val_samples + test_samples
    })

stats_df = pd.DataFrame(stats)
print(stats_df)

# Visualisasi distribusi data
plt.figure(figsize=(12, 6))
stats_df.sort_values('Total', ascending=False).plot(
    x='Spesies', 
    y=['Train', 'Validation', 'Test'], 
    kind='bar', 
    stacked=True
)
plt.title('Distribusi Data per Spesies')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('data_distribution.png')
plt.show()

## 2. Preprocessing Data dan Augmentasi
Dengan TensorFlow, kita dapat menggunakan ImageDataGenerator untuk preprocessing dan augmentasi data.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Ukuran gambar dan batch
img_width, img_height = 224, 224
batch_size = 32

# Augmentasi data untuk mengatasi ketidakseimbangan kelas dan meningkatkan generalisasi
train_datagen = ImageDataGenerator(
    rescale=1./255,               # Normalisasi
    rotation_range=20,            # Rotasi hingga 20 derajat
    width_shift_range=0.2,        # Geser horizontal
    height_shift_range=0.2,       # Geser vertikal
    shear_range=0.2,              # Memiringkan gambar
    zoom_range=0.2,               # Zoom in/out
    horizontal_flip=True,         # Flip horizontal
    brightness_range=[0.8, 1.2],  # Variasi kecerahan
    fill_mode='nearest'           # Strategi mengisi area kosong
)

# Hanya rescaling untuk validasi dan test
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'  # One-hot encoding dilakukan otomatis
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Untuk evaluasi, jangan shuffle data
)

## 3. Feature Engineering
Untuk CNN, feature engineering terjadi secara implisit dalam arsitektur model. Tetapi kita masih perlu mengatasi ketidakseimbangan kelas:

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Class weighting untuk mengatasi ketidakseimbangan kelas
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

# Melihat distribusi class weights
weights_df = pd.DataFrame({
    'Class': classes,
    'Weight': class_weights
})
print(weights_df.sort_values('Weight', ascending=False))

## 4. Model CNN dengan TensorFlow
Berikut adalah implementasi model CNN. Kita akan mencoba dua pendekatan: model custom dari awal dan model transfer learning.

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Model Custom CNN dari awal
def create_custom_cnn():
    model = Sequential([
        # Block 1
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(img_width, img_height, 3)),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Block 2
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Block 3
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Classification block
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Transfer Learning dengan EfficientNet
def create_transfer_learning_model():
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
    
    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Tambahkan custom classifier
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Pilih model yang akan digunakan
model = create_transfer_learning_model()  # Menggunakan transfer learning sebagai model utama
model.summary()

## 5. Melatih Model

In [ ]:
# Setup callbacks untuk training
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    'best_fish_classifier.h5',
    monitor='val_accuracy',
    save_best_only=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=1e-6
)

# Melatih model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint, reduce_lr],
    class_weight=class_weight_dict
)

# Visualisasi proses training
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.savefig('training_history.png')
plt.show()

## 6. Fine-tuning Model
Setelah model dilatih dengan layer dasar yang dibekukan, kita dapat fine-tune model untuk performa yang lebih baik.

In [ ]:
# Unfreeze beberapa layer teratas dari base model
base_model = model.layers[1]  # Mendapatkan base model (EfficientNetB0)

# Unfreeze beberapa layer terakhir
for layer in base_model.layers[-30:]:
    layer.trainable = True
    
# Recompile model dengan learning rate lebih kecil
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Learning rate 10x lebih kecil
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune model
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint, reduce_lr],
    class_weight=class_weight_dict
)

# Visualisasi hasil fine-tuning
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history_fine.history['accuracy'])
plt.plot(history_fine.history['val_accuracy'])
plt.title('Model Accuracy (Fine-tuning)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.title('Model Loss (Fine-tuning)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.savefig('fine_tuning_history.png')
plt.show()

## 7. Evaluasi Model

In [ ]:
# Load model terbaik
model.load_weights('best_fish_classifier.h5')

# Evaluasi pada test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.4f}")

# Prediksi pada test set
test_generator.reset()
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(20, 16))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.savefig('confusion_matrix.png')
plt.show()

# Classification Report
report = classification_report(y_true, y_pred_classes, target_names=classes, output_dict=True)
report_df = pd.DataFrame(report).transpose()
print(report_df)

# Precision, Recall, F1-Score untuk setiap kelas
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred_classes, average=None, labels=np.unique(y_true))

# Visualisasi metrik per kelas
metrics_df = pd.DataFrame({
    'Class': classes,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1
})

# Visualisasi metrik evaluasi
plt.figure(figsize=(12, 8))
metrics_df.set_index('Class').plot(kind='bar')
plt.title('Metrics per Class')
plt.ylabel('Score')
plt.xticks(rotation=90)
plt.legend(loc='lower right')
plt.tight_layout()
plt.savefig('metrics_per_class.png')
plt.show()

# AUC-ROC Curves
plt.figure(figsize=(10, 8))
for i in range(num_classes):
    # One-vs-Rest approach untuk multi-class
    y_true_class = (y_true == i).astype(int)
    y_pred_class = y_pred[:, i]
    
    fpr, tpr, _ = roc_curve(y_true_class, y_pred_class)
    roc_auc = auc(fpr, tpr)
    
    plt.plot(fpr, tpr, lw=2, label=f'{classes[i]} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend(loc="lower right")
plt.tight_layout()
plt.savefig('roc_curves.png')
plt.show()

# Menampilkan metrik keseluruhan
avg_metrics = metrics_df[['Precision', 'Recall', 'F1-Score']].mean()
print("\nMetrik Rata-rata:")
print(avg_metrics)

## 8. Analisis Hasil dan Perbandingan Matriks Evaluasi

In [ ]:
# Function untuk visualisasi dan perbandingan matriks evaluasi
def plot_metric_comparison():
    # Aggregasi metrik
    macro_avg = pd.DataFrame({
        'Precision': [report_df.loc['macro avg', 'precision']],
        'Recall': [report_df.loc['macro avg', 'recall']],
        'F1-Score': [report_df.loc['macro avg', 'f1-score']],
        'Accuracy': [report_df.loc['accuracy', 'precision']]  # accuracy disimpan di lokasi ini di df
    })
    
    # Plot
    plt.figure(figsize=(8, 6))
    ax = macro_avg.plot(kind='bar', color=['blue', 'green', 'red', 'purple'])
    
    # Tambahkan label nilai di atas bar
    for i, v in enumerate(macro_avg.values[0]):
        ax.text(i, v + 0.01, f'{v:.2f}', ha='center')
    
    plt.title('Perbandingan Metrik Evaluasi')
    plt.ylabel('Score')
    plt.ylim(0, 1.1)
    plt.tight_layout()
    plt.savefig('metric_comparison.png')
    plt.show()
    
    # Penjelasan
    print("\nPerbandingan Metrik Evaluasi:")
    print("-----------------------------")
    print(f"Accuracy: {macro_avg['Accuracy'][0]:.4f} - Proporsi prediksi yang benar dari seluruh prediksi")
    print(f"Precision: {macro_avg['Precision'][0]:.4f} - Seberapa akurat model saat memprediksi kelas positif")
    print(f"Recall: {macro_avg['Recall'][0]:.4f} - Seberapa lengkap model mengidentifikasi semua kelas positif")
    print(f"F1-Score: {macro_avg['F1-Score'][0]:.4f} - Harmonic mean dari precision dan recall")
    
    # Menentukan metrik terbaik
    best_metric = ""
    if max(macro_avg.iloc[0]) == macro_avg['Accuracy'][0]:
        best_metric = "Accuracy"
    elif max(macro_avg.iloc[0]) == macro_avg['Precision'][0]:
        best_metric = "Precision"
    elif max(macro_avg.iloc[0]) == macro_avg['Recall'][0]:
        best_metric = "Recall"
    else:
        best_metric = "F1-Score"
    
    print(f"\nMetrik terbaik adalah {best_metric} dengan nilai {max(macro_avg.iloc[0]):.4f}")
    print(f"\nUntuk dataset klasifikasi ikan dengan {num_classes} kelas yang tidak seimbang:")
    
    if best_metric == "Accuracy":
        print("Accuracy merupakan metrik terbaik, menunjukkan model memiliki tingkat kebenaran prediksi yang tinggi secara keseluruhan.")
    elif best_metric == "Precision":
        print("Precision merupakan metrik terbaik, menunjukkan model sangat akurat saat mengidentifikasi spesies ikan tertentu (minim false positive).")
    elif best_metric == "Recall":
        print("Recall merupakan metrik terbaik, menunjukkan model sangat baik dalam mendeteksi semua instance dari spesies ikan tertentu (minim false negative).")
    else:
        print("F1-Score merupakan metrik terbaik, menunjukkan model memiliki keseimbangan yang baik antara precision dan recall, ideal untuk dataset tidak seimbang.")

# Jalankan analisis
plot_metric_comparison()

## 10. Analisis dan Pembahasan Fenomena dalam Deep Learning
Fenomena Vanishing Gradient
Fenomena vanishing gradient terjadi ketika gradien menjadi sangat kecil saat diback-propagate dari lapisan akhir ke lapisan awal, menyebabkan lapisan awal jaringan hampir tidak belajar.

In [ ]:
# Visualisasi Vanishing Gradient dengan Plot Distribusi Gradien
def plot_gradient_distribution(model, train_generator):
    # Membuat model sementara dengan output gradien
    gradient_model = tf.keras.Model(
        inputs=model.inputs,
        outputs=[layer.output for layer in model.layers if isinstance(layer, tf.keras.layers.Conv2D)]
    )
    
    # Mendapatkan batch data
    x_batch, y_batch = next(train_generator)
    
    # Hitung gradien
    with tf.GradientTape() as tape:
        activations = gradient_model(x_batch)
        loss = tf.keras.losses.categorical_crossentropy(y_batch, model.predict(x_batch))
    
    # Mendapatkan gradien untuk setiap lapisan konvolusi
    grads = tape.gradient(loss, activations)
    
    # Visualisasi distribusi gradien untuk setiap lapisan
    plt.figure(figsize=(12, 8))
    for i, grad in enumerate(grads):
        plt.subplot(len(grads), 1, i+1)
        plt.hist(tf.keras.backend.flatten(grad).numpy(), bins=50)
        plt.title(f'Layer {i+1} Gradient Distribution')
        plt.xlabel('Gradient Value')
        plt.ylabel('Frequency')
    
    plt.tight_layout()
    plt.savefig('gradient_distribution.png')
    plt.show()
    
    # Analisis
    print("Analisis Vanishing Gradient:")
    for i, grad in enumerate(grads):
        grad_mean = np.mean(np.abs(grad.numpy()))
        print(f"Layer {i+1} - Mean Gradient Magnitude: {grad_mean:.8f}")
        
    print("\nSolusi untuk Vanishing Gradient:")
    print("1. Gunakan skip connections (ResNet)")
    print("2. Batch Normalization sebelum aktivasi")
    print("3. Ganti ReLU dengan Leaky ReLU atau ELU")
    print("4. Gunakan inisialisasi bobot yang tepat (He initialization)")
    print("5. Gunakan Gradient Clipping")

# Visualisasi gradien setelah beberapa epoch training
# plot_gradient_distribution(model, train_generator)

**Fenomena ReLU Dying dan Strategi Mengatasinya**
**Fenomena dying ReLU terjadi ketika neuron ReLU secara konsisten menghasilkan output 0.**

In [ ]:
# Implementasi dan perbandingan berbagai aktivasi untuk mengatasi dying ReLU
def create_activation_comparison_model(activation='relu'):
    model = Sequential([
        Conv2D(32, (3, 3), padding='same', input_shape=(img_width, img_height, 3)),
        
        # Pilih aktivasi berdasarkan parameter
        tf.keras.layers.Activation(activation) if activation != 'leaky_relu' and activation != 'prelu' 
        else tf.keras.layers.LeakyReLU(alpha=0.1) if activation == 'leaky_relu'
        else tf.keras.layers.PReLU(),
        
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), padding='same'),
        
        # Pilih aktivasi lagi
        tf.keras.layers.Activation(activation) if activation != 'leaky_relu' and activation != 'prelu' 
        else tf.keras.layers.LeakyReLU(alpha=0.1) if activation == 'leaky_relu'
        else tf.keras.layers.PReLU(),
        
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128),
        
        # Pilih aktivasi lagi
        tf.keras.layers.Activation(activation) if activation != 'leaky_relu' and activation != 'prelu' 
        else tf.keras.layers.LeakyReLU(alpha=0.1) if activation == 'leaky_relu'
        else tf.keras.layers.PReLU(),
        
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Untuk percobaan: bandingkan berbagai fungsi aktivasi untuk mengatasi dying ReLU
# aktivasi = ['relu', 'leaky_relu', 'prelu', 'elu', 'selu']
# histories = {}
# for act in aktivasi:
#     print(f"\nTraining model dengan aktivasi {act}...")
#     model = create_activation_comparison_model(act)
#     histories[act] = model.fit(
#         train_generator,
#         steps_per_epoch=train_generator.samples // (batch_size * 10),  # Kurangi untuk percobaan
#         epochs=10,  # Kurangi untuk percobaan
#         validation_data=validation_generator,
#         validation_steps=validation_generator.samples // (batch_size * 10)  # Kurangi untuk percobaan
#     ).history
#
# # Visualisasi perbandingan
# plt.figure(figsize=(12, 5))
# plt.subplot(1, 2, 1)
# for act in aktivasi:
#     plt.plot(histories[act]['accuracy'], label=act)
# plt.title('Training Accuracy per Activation')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend()
#
# plt.subplot(1, 2, 2)
# for act in aktivasi:
#     plt.plot(histories[act]['val_accuracy'], label=act)
# plt.title('Validation Accuracy per Activation')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend()
#
# plt.tight_layout()
# plt.savefig('activation_comparison.png')
# plt.show()

**Fenomena Overfitting dan Strategi Regularisasi**

In [ ]:
# Function untuk mendemonstrasikan overfitting dan efek regularisasi
def demonstrate_overfitting(model, train_generator, validation_generator, epochs=50):
    # Training tanpa regularisasi
    history_no_reg = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size,
    )
    
    # Tambahkan regularisasi L2 ke model
    for layer in model.layers:
        if hasattr(layer, 'kernel_regularizer'):
            layer.kernel_regularizer = tf.keras.regularizers.l2(0.001)
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Training dengan regularisasi
    history_with_reg = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size,
    )
    
    # Visualisasi
    plt.figure(figsize=(12, 10))
    
    plt.subplot(2, 2, 1)
    plt.plot(history_no_reg.history['accuracy'])
    plt.plot(history_no_reg.history['val_accuracy'])
    plt.title('Model Accuracy (No Regularization)')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    plt.subplot(2, 2, 2)
    plt.plot(history_no_reg.history['loss'])
    plt.plot(history_no_reg.history['val_loss'])
    plt.title('Model Loss (No Regularization)')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    plt.subplot(2, 2, 3)
    plt.plot(history_with_reg.history['accuracy'])
    plt.plot(history_with_reg.history['val_accuracy'])
    plt.title('Model Accuracy (With L2 Regularization)')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    plt.subplot(2, 2, 4)
    plt.plot(history_with_reg.history['loss'])
    plt.plot(history_with_reg.history['val_loss'])
    plt.title('Model Loss (With L2 Regularization)')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    plt.tight_layout()
    plt.savefig('overfitting_demonstration.png')
    plt.show()
    
    # Analisis
    print("\nFenomena Overfitting:")
    print("-------------------")
    print("Tanpa Regularisasi:")
    print(f"Final Training Accuracy: {history_no_reg.history['accuracy'][-1]:.4f}")
    print(f"Final Validation Accuracy: {history_no_reg.history['val_accuracy'][-1]:.4f}")
    print(f"Gap: {history_no_reg.history['accuracy'][-1] - history_no_reg.history['val_accuracy'][-1]:.4f}")
    
    print("\nDengan Regularisasi L2:")
    print(f"Final Training Accuracy: {history_with_reg.history['accuracy'][-1]:.4f}")
    print(f"Final Validation Accuracy: {history_with_reg.history['val_accuracy'][-1]:.4f}")
    print(f"Gap: {history_with_reg.history['accuracy'][-1] - history_with_reg.history['val_accuracy'][-1]:.4f}")
    
    print("\nStrategi untuk Mengatasi Overfitting:")
    print("1. Regularisasi L1/L2")
    print("2. Dropout")
    print("3. Data Augmentation")
    print("4. Early Stopping")
    print("5. Batch Normalization")

# Demonstrasi overfitting dapat dijalankan dengan:
# model_demo = create_custom_cnn()
# demonstrate_overfitting(model_demo, train_generator, validation_generator, epochs=20)